# Run BASIS

## Setup Env

In [1]:
import os, fnmatch

import numpy as np
import pandas as pd

import scanpy as sc
import squidpy as sq
import mudata as mu

In [2]:
import liana as li
from liana.method.sp import basis

In [3]:
import decoupler as dc
import seaborn as sns

In [6]:
data_dir = "../../data/kidney_injury/visium"

In [7]:
# consider making this compatible to squidpy params (e.g. bypass_diagonal -> set_diag)
# li.mt.get_spatial_proximity(adata=mdata, parameter=200, bypass_diagonal=False, cutoff=0.2)

## Load Processed Objects

### Load Visium

In [8]:
slides = fnmatch.filter(os.listdir(os.path.join(data_dir, 'mdatas')), '*.h5ad')

In [9]:
mdatas = dict()
for slide in slides:
    sample = slide[:-5].split('_')[0]
    print(sample)
    
    mdata = mu.read_h5mu(os.path.join(data_dir, 'mdatas', sample + '.h5ad'))
    mdata.obsm['spatial'] = mdata.mod['rna'].obsm['spatial']
    mdata.uns['spatial'] = mdata.mod['rna'].uns['spatial']
    
    sq.gr.spatial_neighbors(mdata, n_rings=1, coord_type="grid", n_neighs=6, set_diag=True)
    
    mdatas[sample] = mdata
    

f12hr


/home/dbdimitrov/anaconda3/envs/spiana/lib/python3.10/site-packages/mudata/_core/mudata.py:458: UserWarning: Cannot join columns with the same name because var_names are intersecting.


f2dps


/home/dbdimitrov/anaconda3/envs/spiana/lib/python3.10/site-packages/mudata/_core/mudata.py:458: UserWarning: Cannot join columns with the same name because var_names are intersecting.


f4hr


/home/dbdimitrov/anaconda3/envs/spiana/lib/python3.10/site-packages/mudata/_core/mudata.py:458: UserWarning: Cannot join columns with the same name because var_names are intersecting.


f6wks


/home/dbdimitrov/anaconda3/envs/spiana/lib/python3.10/site-packages/mudata/_core/mudata.py:458: UserWarning: Cannot join columns with the same name because var_names are intersecting.


fsham


/home/dbdimitrov/anaconda3/envs/spiana/lib/python3.10/site-packages/mudata/_core/mudata.py:458: UserWarning: Cannot join columns with the same name because var_names are intersecting.


## Basis

Ligand to Cytokine Signatures

In [ ]:
# load map of genes to cytosig signatures
cytosig_map = pd.read_csv('../cytosig_map.csv')
# Translate targets
cytosig_map = dc.translate_net(cytosig_map, target_organism = 'mouse', columns=['gene'])
cytosig_map.head()

In [ ]:
# Cytokine expr to Cytosig Siagntures interactions
interactions = list(zip(cytosig_map['gene'], cytosig_map['signature']))

Receptor to TF

In [ ]:
# interaction to Dorothea TFs links
receptor_tf = pd.read_csv("../../data/receptor_tf.csv")
receptor_tf = receptor_tf[['interaction', 'target']].drop_duplicates()

## TODO change basis lr_sep to `^`
receptor_tf['interaction'] = receptor_tf['interaction'].str.replace('^', '&', regex=False)


receptor_tf_links = list(zip(receptor_tf['interaction'], receptor_tf['target']))

## Run Basis on every sample

In [ ]:
for sample in mdatas.keys():
      
      mdata = mdatas[sample]
      
      # ligand -> Cytokine
      basis(mdata, function_name='pearson', 
            x_mod='rna',
            y_mod='cytosig',
            interactions=interactions,
            proximity_key='spatial_connectivities',
            mod_added = 'lig_cytosig'
            )
      
      # LR -> TF
      basis(mdata,
            function_name='cosine', # morans sucks
            x_mod='lrs',
            y_mod='dorothea',
            interactions=receptor_tf_links,
            proximity_key='spatial_connectivities',
            mod_added = 'lrs_tf',
            key_added = 'lr_tf_global'
            )
      
      # PROGENy -> NMF      
      basis(mdata, function_name='cosine', 
            x_mod='progeny',
            y_mod='nmf',
            proximity_key='spatial_connectivities',
            mod_added = 'progeny_nmf'
            )
      
      # Tangram -> NMF      
      basis(mdata, function_name='spearman', 
            x_mod='tangram',
            y_mod='nmf',
            proximity_key='spatial_connectivities',
            mod_added = 'tangram_nmf'
            )
      

## Ligand to Signature

In [ ]:
for sample in mdatas.keys():
    cyto = mdatas[sample].mod['lig_cytosig']
    
    # names = cyto.var_names[cyto.var_names.str.contains('TGFB1')]
    names = cyto.var_names
    
    print(sample)
    sc.pl.spatial(cyto, color=names, cmap='coolwarm', vmax=1, vmin=-1) #, title=sample)

## Interaction to TF

##### get top 10 most correlated LR->TFs for each sample

In [ ]:
top_interactions = list()

nmf_vars = dict()
factor_names = mdatas['f12hr'].mod['nmf'].var.index

for sample in mdatas.keys():
    global_res = mdatas[sample].mod['lig_cytosig'].uns['lr_tf_global']
    global_res = global_res.sort_values('global_sd', ascending=False).head(5)
    top_interactions.append(global_res[['interaction']])
    
    

In [ ]:
top_interactions = np.unique(top_interactions)
top_interactions

In [ ]:
# plot top interactions
for sample in mdatas.keys():
    lrs_tf = mdatas[sample].mod['lrs_tf']
    
    top = top_interactions[np.isin(top_interactions, lrs_tf.var_names)]
    
    print(sample)
    sc.pl.spatial(lrs_tf, color=top, cmap='coolwarm') #, vmin=-1, title=sample)

##### alternatively focus only on interactions in the factor specific to the recovery

In [ ]:
lrdata = sc.read_h5ad(os.path.join("../../data/kidney_injury/lrs_all_factors.h5ad"))

In [ ]:
factor_names = mdatas['f12hr'].mod['nmf'].var.index

In [ ]:
lr_loadings = pd.DataFrame(lrdata.varm['NMF_H'], index=lrdata.var.index, columns=factor_names)

In [ ]:
lr_loadings.to_csv(os.path.join("../../data/kidney_injury/lr_loadings.csv"))

In [ ]:
top_loadings = lr_loadings.sort_values('Factor3', ascending=False).head(20).index

In [ ]:
# plot top interactions
from functools import reduce
for sample in mdatas.keys():
    lrs_tf = mdatas[sample].mod['lrs_tf']
    
    top = reduce(np.union1d, [lrs_tf.var_names[lrs_tf.var_names.str.contains(lr)] for lr in top_loadings])
    
    print(sample)
    sc.pl.spatial(lrs_tf, color=top, cmap='coolwarm') #, vmin=-1, title=sample)

## NMF to PROGENy

In [ ]:
for sample in mdatas.keys():
    pronmf = mdatas[sample].mod['progeny_nmf']
    
    names = pronmf.var_names[pronmf.var_names.str.contains('Factor8')]
    
    print(sample)
    sc.pl.spatial(pronmf, color=names, cmap='coolwarm') #, title=sample)

## Deconv props x NFM

In [ ]:
for sample in mdatas.keys():
    view = mdatas[sample].mod['tangram_nmf']
    
    names = view.var_names[view.var_names.str.contains('Factor3')]
    
    print(sample)
    sc.pl.spatial(view, color=names, cmap='cividis') #, title=sample)